In [1]:
# 基本配置
from langchain_openai import ChatOpenAI
import os
from dotenv import load_dotenv

load_dotenv(override=True)

qw_llm_openai = ChatOpenAI(
    openai_api_base=os.getenv('DASHSCOPE_API_BASE'),
    openai_api_key=os.getenv('DASHSCOPE_API_KEY'),
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
)

In [2]:
from langchain_community.embeddings.cloudflare_workersai import CloudflareWorkersAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()
embedding = CloudflareWorkersAIEmbeddings(
    account_id=os.getenv('CF_ACCOUNT_ID'),
    api_token=os.getenv('CF_API_TOKEN'),
    model_name="@cf/baai/bge-small-en-v1.5",
)

In [4]:
from langchain.chains.retrieval import create_retrieval_chain
import bs4
# from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embedding)
retriever = vectorstore.as_retriever()

# 2. Incorporate the retriever into a question-answering chain.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
    "\n\n"
    "Previous conversation:\n{history}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(qw_llm_openai, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
from langchain.memory import ConversationBufferMemory

## rag + llm

In [3]:
from typing import Optional, Dict
from langchain_core.runnables.utils import Input
from langchain_core.runnables import Runnable, RunnableConfig
from langchain_core.load import Serializable


# 自定义一个继承Runnable的类
class StdOutputRunnable(Serializable, Runnable[Input, Input]):
    @property
    def lc_serializable(self) -> bool:
        return True

    def invoke(self, input: Dict, config: Optional[RunnableConfig] = None) -> Input:
        # print(f"Hey, I received the name {input['name']}")
        print(input)
        return self._call_with_config(lambda x: x, input, config)

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


# 这个函数接收一个文档列表，并将它们的页面内容（page_content）合并成一个单一的字符串，每个文档之间用两个换行符分隔
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# 执行流程2:
# rag_chain_from_docs 使用 retrieve_docs 的输出（问题）和 format_docs 函数来格式化检索到的文档内容。
rag_chain_from_docs = (
    # 这一步接收 retrieve_docs 函数的输出，将其作为 context 传递给 format_docs 函数，生成格式化的文档内容字符串。
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["retriever_context"])))
        | prompt
        | qw_llm_openai
        | StrOutputParser()
)

# 执行流程1: 从字典中提取 input 键的值，即 "What is Task Decomposition"。用户的问题
# retrieve_docs 函数的输出是一个包含 context 键的字典，那么 x 就会包含这个键
retrieve_docs = (lambda x: x["input"]) | retriever
# retrieve_docs = (lambda x: {"retriever_context": retriever.get_relevant_documents(x["input"])})

# 组合
# chain：这个变量组合了两个步骤，
# 首先通过 retrieve_docs 获取问题，
# 然后通过 rag_chain_from_docs 来生成答案。
# retriever_context会被传递到下一个步骤里去
chain = RunnablePassthrough.assign(retriever_context=retrieve_docs).assign(
    answer=rag_chain_from_docs
)

In [15]:

# 详细流程：
# 1. 从字典中提取 input 键的值，即 "What is Task Decomposition"。用户的问题
# 2. 使用 retrieve_docs 函数来检索文档，并把结果赋值给 retriever_context，再传递给下一个步骤（即rag_chain_from_docs）
# 3. rag_chain_from_docs 使用 retrieve_docs 的输出（问题）和 format_docs 函数来格式化检索到的文档内容。
chain.invoke({"input": "What is Task Decomposition"})

{'input': 'What is Task Decomposition',
 'retriever_context': [Document(page_content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}),
  Document(page_content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts

## rag + memory + llm

In [2]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain.memory.chat_message_histories import FileChatMessageHistory

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
    "\n\n"
    "Previous conversation:\n{history}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# memory1
# memory = ConversationBufferMemory(return_messages=True)

# memory2
path = '../../../data/history/conversation_20240709-2.json'
message_history = FileChatMessageHistory(file_path=path)
# memory = ConversationBufferMemory(chat_memory=message_history, return_messages=True)

# memory3
memory = ConversationBufferWindowMemory(k=2, chat_memory=message_history, return_messages=True)

rag_chain_from_docs = (
        RunnablePassthrough.assign(
            context=(lambda x: format_docs(x["retriever_context"])),
            history=memory.load_memory_variables
        )
        | prompt
        | qw_llm_openai
        | StrOutputParser()
)
retrieve_docs = (lambda x: x["input"]) | retriever 
# retrieve_docs = lambda x: {"retriever_context": retriever.get_relevant_documents(x["input"])}

chain = (
    RunnablePassthrough.assign(retriever_context=retrieve_docs)
    .assign(answer=rag_chain_from_docs)
    .assign(
        memory_update=lambda x: memory.save_context(
            {"input": x["input"]},
            {"output": x["answer"]}
        )
    )
)


# 使用示例
def chat(input_text):
    result = chain.invoke({"input": input_text})
    return result["answer"]

# 使用方法
# response = chat("What is Task Decomposition?")
# print(response)
# 继续对话
# response = chat("Can you give me an example?")
# print(response)

NameError: name 'qw_llm_openai' is not defined

In [42]:
response = chat("What is Task Decomposition?")
print(response)

Task Decomposition is a method used in artificial intelligence where a large task is broken down into smaller, more manageable parts called subtasks. This approach helps the AI system understand the task better and perform it more efficiently.


In [43]:
response = chat("Can you give me an example?")
print(response)

{
    "thoughts": {
        "text": "The user wants an example of task decomposition.",
        "reasoning": "I will provide an example of how task decomposition works in a simple scenario.",
        "plan": "- Provide an example of task decomposition\n- Explain how it breaks down the task into smaller parts\n- Suggest that the user provides feedback on whether they understood the concept",
        "criticism": "It's important to ensure the explanation is clear and understandable.",
        "speak": "Sure, here's an example: Imagine you're organizing a party. You have a list of tasks to complete: buy decorations, invite guests, prepare food, etc. By breaking this task into smaller sub-tasks such as buying decorations, inviting guests, and preparing food, you can manage each part of the event more effectively."
    },
    "command": {
        "name": "command name",
        "args": {
            "arg name": "value"
        }
    }
}


In [45]:
response = chat("我刚才问了什么,hhhh?")
print(response)

你之前问的问题是关于"Task Decomposition"的概念。


In [31]:
memory.load_memory_variables

<bound method ConversationBufferMemory.load_memory_variables of ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='What is Task Decomposition?'), AIMessage(content='Task Decomposition is a method used in artificial intelligence where a large task is broken down into smaller, more manageable parts called subtasks. This approach helps the AI system understand the task better and perform it more efficiently.'), HumanMessage(content='Can you give me an example?'), AIMessage(content='{\n    "thoughts": {\n        "text": "The user wants an example of task decomposition.",\n        "reasoning": "I will provide an example of how task decomposition works in a simple scenario.",\n        "plan": "- Provide an example of task decomposition\\n- Explain how it breaks down the task into smaller parts\\n- Suggest that the user provides feedback on whether they understood the concept",\n        "criticism": "It\'s important to ensure the explanation is cle

In [36]:
memory.chat_memory.messages

[HumanMessage(content='What is Task Decomposition?'),
 AIMessage(content='Task Decomposition is a method used in artificial intelligence where a large task is broken down into smaller, more manageable parts called subtasks. This approach helps the AI system understand the task better and perform it more efficiently.'),
 HumanMessage(content='Can you give me an example?'),
 AIMessage(content='{\n    "thoughts": {\n        "text": "The user wants an example of task decomposition.",\n        "reasoning": "I will provide an example of how task decomposition works in a simple scenario.",\n        "plan": "- Provide an example of task decomposition\\n- Explain how it breaks down the task into smaller parts\\n- Suggest that the user provides feedback on whether they understood the concept",\n        "criticism": "It\'s important to ensure the explanation is clear and understandable.",\n        "speak": "Sure, here\'s an example: Let\'s break down the task of cooking a meal."\n    },\n    "com

### 方式2 RunnableWithMessageHistory

In [34]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_from_docs = (
        RunnablePassthrough.assign(context=(lambda x: format_docs(x["retriever_context"])))
        | prompt
        | qw_llm_openai
        | StrOutputParser()
)

retrieve_docs = (lambda x: x["input"]) | retriever
# retrieve_docs = lambda x: {"retriever_context": retriever.get_relevant_documents(x["input"])}

chain = (
    RunnablePassthrough.assign(retriever_context=retrieve_docs)
    .assign(answer=rag_chain_from_docs)
)


def get_session_history():
    return ChatMessageHistory()


chain_with_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)


# 使用示例
def chat(input_text, session_id):
    result = chain_with_history.invoke(
        {"input": input_text},
        config={"configurable": {"session_id": session_id}}
    )
    return result["answer"]

# 使用方法
# response = chat("What is Task Decomposition?", "user_1")
# print(response)
# 继续对话
# response = chat("Can you give me an example?", "user_1")
# print(response)

In [35]:
response = chat("What is Task Decomposition?", "user_1")
print(response)

TypeError: get_session_history() takes 0 positional arguments but 1 was given

In [49]:
# 假设 content 是一个包含 Unicode 编码的字符串
content = b'\u725b\u80cc\u5c71\u4f60\u77e5\u9053\u5417'

# 使用 decode 方法将编码的字符串转换为正常的中文字符
decoded_content = content.decode('unicode_escape')

print(decoded_content)

牛背山你知道吗


In [50]:
# 假设 byte_content 是从外部获取的字节序列
byte_content = b'\xe7\x88\x86\xe8\x99\x8e\xe5\xb1\xb1\xe4\xbd\xa0\xe7\x9f\xa5\xe9\x81\x93\xe5\x90\x97'

# 使用 decode 方法将字节序列解码为字符串
content = content.decode('utf-8')

print(content)

\u725b\u80cc\u5c71\u4f60\u77e5\u9053\u5417
